In [10]:
import pandas as pd
import numpy as np

In [11]:
# Import WeatherPy Database CSV
weatherpy_database_df = pd.read_csv("weather_data/WeatherPy_database.csv")
weatherpy_database_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rainfall(1hr),Snowfall(1hr)
0,0,Komsomolskiy,UZ,40.43,71.72,75.20,41,0,6.93,clear sky,0,0
1,1,Barrow,US,71.29,-156.79,46.40,75,1,25.28,clear sky,0,0
2,2,Kudahuvadhoo,MV,2.67,72.89,82.98,76,74,15.57,light rain,0,0
3,3,Yellowknife,CA,62.46,-114.35,73.40,46,40,10.29,scattered clouds,0,0
4,4,Tuatapere,NZ,-46.13,167.68,54.55,70,100,7.99,overcast clouds,0,0


In [12]:
# Create a DataFrame for each of the selected cities.
botwood_df = weatherpy_database_df.loc[(weatherpy_database_df["City"]=="Botwood")]
torbay_df = weatherpy_database_df.loc[(weatherpy_database_df["City"]=="Torbay")]
grand_bank_df = weatherpy_database_df.loc[(weatherpy_database_df["City"]=="Grand Bank")]
glace_bay_df = weatherpy_database_df.loc[(weatherpy_database_df["City"]=="Glace Bay")]

In [13]:
#  Combine the four DataFrames and add hotel search.
vacation_itinerary_df = pd.concat([botwood_df, torbay_df, grand_bank_df, glace_bay_df])
vacation_itinerary_df["Hotel Name"] = ""

vacation_itinerary_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rainfall(1hr),Snowfall(1hr),Hotel Name
202,202,Botwood,CA,49.15,-55.35,62.01,82,89,3.44,overcast clouds,0,0,
308,308,Torbay,CA,47.67,-52.73,55.00,100,90,6.93,overcast clouds,0,0,
389,389,Grand Bank,CA,47.10,-55.77,57.51,89,92,14.38,overcast clouds,0,0,
258,258,Glace Bay,CA,46.20,-59.96,68.00,94,75,4.70,broken clouds,0,0,


In [14]:
import gmaps
import gmaps.datasets
import requests
from config import g_key

In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in vacation_itinerary_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        vacation_itinerary_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
vacation_itinerary_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rainfall(1hr),Snowfall(1hr),Hotel Name
202,202,Botwood,CA,49.15,-55.35,62.01,82,89,3.44,overcast clouds,0,0,Exploits Hostel
308,308,Torbay,CA,47.67,-52.73,55.00,100,90,6.93,overcast clouds,0,0,Gallows Cove
389,389,Grand Bank,CA,47.10,-55.77,57.51,89,92,14.38,overcast clouds,0,0,Hotel Fortune
258,258,Glace Bay,CA,46.20,-59.96,68.00,94,75,4.70,broken clouds,0,0,Cedarwoods Bed & Breakfast


In [17]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in vacation_itinerary_df.iterrows()]


In [18]:
# Latitude-longitude pairs
botwood = (49.15, -55.35)
torbay = (47.67, -52.73)
grand_bank = (47.1, -55.77)
glace_bay = (46.2, -59.96)

# Create the figure.
fig = gmaps.figure()

#Add markers
locations = vacation_itinerary_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Add navigation.
botwood2glace_bay = gmaps.directions_layer(botwood, torbay, travel_mode='DRIVING')
fig.add_layer(botwood2glace_bay)




fig

Figure(layout=FigureLayout(height='420px'))